Prerequisites and Notes

Installation: The error likely stems from an incomplete/outdated timesfm package. Install from source (editable mode) as per the repo:

```bash 
git clone https://github.com/google-research/timesfm.git
cd timesfm
pip install -e .
```

In [1]:
# Cell 2: Imports
# Ensure all available; from finetuning_torch.py and notebook
import os
from typing import Tuple

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
import yfinance as yf  # For external data augmentation if needed
import wandb  # Optional for logging

from timesfm import TimesFmCheckpoint, TimesFmHparams
from timesfm.timesfm_torch import TimesFmTorch
from timesfm.pytorch_patched_decoder import PatchedTimeSeriesDecoder, create_quantiles

# From finetuning_torch.py (assume in path or copy classes here if modularizing later)
from finetuning.finetuning_torch import (
    MetricsLogger, WandBLogger, DistributedManager,
    FinetuningConfig, TimesFMFinetuner
)

# Project-specific (adapt paths; from your src/)
from dataprep import dataprep  # Your data prep class
from configs import TrainConfig  # If needed for params

print(f"CUDA available: {torch.cuda.is_available()}") 

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.13 | packaged by conda-forge | (main, Jun  4 2025, 14:48:23) [GCC 13.3.0].
CUDA available: True


/root/anaconda3/envs/mitsui/lib/python3.11/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
# Cell 3: Data Preparation - Load and Preprocess Project Data
# Use dataprep.py to load; melt targets for univariate/multi-target handling
# Augment with relevant prices (yfinance for LME/CME-like; offline in competition)

# Initialize data preparer (from your dataprep.py)
data_path = '../data'  # Update to your train.csv, train_labels.csv
data_preparer = dataprep(data_path)
data_preparer.one_shot_prep()

# Targets: 424 series from train_labels.csv
Y_full = data_preparer.Y_full_np  # (N~600, T=424)
#dates = data_preparer.dates  # Or derive from date_id

# For univariate: Treat each target column as separate series
# Shape: List of 424 arrays, each (N,)
univariate_series = [Y_full[:, i] for i in range(424)]

# For multivariate: Incorporate features from train.csv (X_full_np: N, F~558)
# Scalarize per target: e.g., mean of all features + target history (univariate proxy)
X_full = data_preparer.X_full_np  # (N, F)
multivariate_series = []
for i in range(424):
    target_hist = Y_full[:, i]
    feat_mean = np.nanmean(X_full, axis=1)  # Or PCA/project: simplify to mean for demo
    combined = np.vstack([target_hist, feat_mean]).T  # (N, 2); project to univariate if needed
    combined_scalar = np.mean(combined, axis=1)  # Scalarize to univariate (N,)
    multivariate_series.append(combined_scalar)

# Choose mode: univariate or multivariate
use_multivariate = False  # Toggle True for multivariate
series_list = multivariate_series if use_multivariate else univariate_series

print(f"Prepared {len(series_list)} series; each shape: {series_list[0].shape}")

2025-10-02 15:28:16,175 - INFO - Loading train.csv from ../data/train.csv
2025-10-02 15:28:16,277 - INFO - Loading train_labels.csv from ../data/train_labels.csv
2025-10-02 15:28:16,399 - INFO - Loading target_pairs.csv from ../data/target_pairs.csv
2025-10-02 15:28:16,552 - INFO - X_full NaNs after imputation: 0
2025-10-02 15:28:16,553 - INFO - Y_full NaNs before imputation: 0
2025-10-02 15:28:16,555 - INFO - Y_full NaNs after imputation: 0
2025-10-02 15:28:16,555 - INFO - X_full feature count: 1671
2025-10-02 15:28:16,564 - INFO - Data loaded: X_full_np shape=(1917, 1671), Y_full_np shape=(1917, 424)


Prepared 424 series; each shape: (1917,)


In [3]:
# Cell 4: Create TimeSeriesDataset for Fine-Tuning
# Ensures horizon_length=1 for single-step prediction

import torch
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    """Dataset for time series data compatible with TimesFM."""
    def __init__(self, series: np.ndarray, context_length: int, horizon_length: int = 1, freq_type: int = 0):
        if freq_type not in [0, 1, 2]:
            raise ValueError("freq_type must be 0, 1, or 2")
        self.series = np.asarray(series, dtype=np.float32)
        self.context_length = context_length
        self.horizon_length = horizon_length
        self.freq_type = freq_type
        self._prepare_samples()

    def _prepare_samples(self) -> None:
        self.samples = []
        total_length = self.context_length + self.horizon_length
        for start_idx in range(0, len(self.series) - total_length + 1):
            end_idx = start_idx + self.context_length
            x_context = self.series[start_idx:end_idx]
            x_future = self.series[end_idx:end_idx + self.horizon_length]  # (horizon_length=1,)
            if np.all(np.isfinite(x_context)) and np.all(np.isfinite(x_future)):
                self.samples.append((x_context, x_future))
            else:
                print(f"Skipping invalid sample at {start_idx}")

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        x_context, x_future = self.samples[index]
        x_context = torch.tensor(x_context, dtype=torch.float32)  # (context_length,)
        x_future = torch.tensor(x_future, dtype=torch.float32)  # (horizon_length=1,)
        if x_future.dim() == 1:
            x_future = x_future.unsqueeze(-1)  # (horizon_length=1, 1)
        input_padding = torch.zeros_like(x_context)  # (context_length,)
        freq = torch.tensor([self.freq_type], dtype=torch.long)  # (1,)
        #print(f"Sample {index}: x_future shape {x_future.shape}")
        return x_context, input_padding, freq, x_future

# Params: Match competition (horizon=1, context~480)
context_length = 512  # Up to 8192; use 512 for efficiency
horizon_length = 1    # One-step-ahead
freq_type = 0         # Daily/unknown
train_split = 0.8     # ~480 train, ~120 val per series

datasets = []
for series in series_list:
    train_size = int(len(series) * train_split)
    train_data = series[:train_size]
    val_data = series[train_size:]
    train_dataset = TimeSeriesDataset(train_data, context_length, horizon_length, freq_type)
    val_dataset = TimeSeriesDataset(val_data, context_length, horizon_length, freq_type)
    datasets.append((train_dataset, val_dataset))

print(f"Created {len(datasets)} datasets; sample train len: {len(datasets[0][0])}, val len: {len(datasets[0][1])}")

Created 424 datasets; sample train len: 1021, val len: 0


In [4]:
# Cell 5: Initialize Model and Fine-Tuner
# Load TimesFM v1.0-200m (foundational v1); fine-tune with FinetuningConfig/TimesFMFinetuner

import timesfm
from timesfm import TimesFmHparams, TimesFmCheckpoint
from timesfm.pytorch_patched_decoder import PatchedTimeSeriesDecoder, create_quantiles
import torch


# Define model hyperparameters
hparams = TimesFmHparams(
    num_layers=20,
    num_heads=16,
    model_dims=1280,
    input_patch_len=32,
    output_patch_len=128,
    quantiles=create_quantiles(),
    use_positional_embedding=True,
    horizon_len=1  # Explicitly set for univariate single-target forecasting
)

# Define checkpoint
checkpoint = TimesFmCheckpoint(
    path = '/root/vscode/portfolio/mitsui_2025/checkpoints/timesfm-1.0-200m/torch_model.ckpt'
)

# Initialize TimesFM model
tfm = TimesFmTorch(
    hparams=hparams,
    checkpoint=checkpoint,
)

# Load pretrained checkpoint
tfm.load_from_checkpoint(checkpoint)

# Fine-tuning config
config = FinetuningConfig(
    batch_size=32,
    num_epochs=50,
    learning_rate=1e-4,
    weight_decay=0.01,
    freq_type=0,
    use_quantile_loss=True,
    quantiles=create_quantiles(),
    device="cpu",
    distributed=False,
    use_wandb=False,
    log_every_n_steps=10,
    val_check_interval=0.5
)

# Initialize finetuner
finetuners = []
for i, (train_ds, val_ds) in enumerate(datasets):
    finetuner = TimesFMFinetuner(
        model=tfm._model,
        config=config,
        rank=0,
        loss_fn=None
    )
    finetuners.append(finetuner)
    print(f"Initialized finetuner for target {i}")


print("Model and finetuners initialized.")

2025-10-02 15:28:22,196 - INFO - Loading checkpoint from /root/vscode/portfolio/mitsui_2025/checkpoints/timesfm-1.0-200m/torch_model.ckpt
2025-10-02 15:28:22,268 - INFO - Sending checkpoint to device cpu
2025-10-02 15:28:24,139 - INFO - Loading checkpoint from /root/vscode/portfolio/mitsui_2025/checkpoints/timesfm-1.0-200m/torch_model.ckpt
2025-10-02 15:28:24,213 - INFO - Sending checkpoint to device cpu


Initialized finetuner for target 0
Initialized finetuner for target 1
Initialized finetuner for target 2
Initialized finetuner for target 3
Initialized finetuner for target 4
Initialized finetuner for target 5
Initialized finetuner for target 6
Initialized finetuner for target 7
Initialized finetuner for target 8
Initialized finetuner for target 9
Initialized finetuner for target 10
Initialized finetuner for target 11
Initialized finetuner for target 12
Initialized finetuner for target 13
Initialized finetuner for target 14
Initialized finetuner for target 15
Initialized finetuner for target 16
Initialized finetuner for target 17
Initialized finetuner for target 18
Initialized finetuner for target 19
Initialized finetuner for target 20
Initialized finetuner for target 21
Initialized finetuner for target 22
Initialized finetuner for target 23
Initialized finetuner for target 24
Initialized finetuner for target 25
Initialized finetuner for target 26
Initialized finetuner for target 27
In

In [ ]:
# Cell 6: Fine-Tuning Loop
# Fine-tune each of the 424 target series; save checkpoints and log metrics
# Ensure finetuning_torch.py is patched for _process_batch
import os
import torch


# Verify patch in finetuning_torch.py
from finetuning.finetuning_torch import TimesFMFinetuner
assert hasattr(TimesFMFinetuner, '_process_batch'), "Patch finetuning_torch.py with updated _process_batch"

# Fine-tune each target
histories = []
for i, (train_ds, val_ds) in enumerate(datasets):
    print(f"Fine-tuning target {i}/{len(datasets)-1}...")
    result = finetuners[i].finetune(train_dataset=train_ds, val_dataset=val_ds, single_step_pred=True)
    histories.append(result['history'])
    
    # Save checkpoint
    save_dir = f"../checkpoints/timesfm/finetuned_target_{i}"
    os.makedirs(save_dir, exist_ok=True)
    torch.save(finetuners[i].model.state_dict(), os.path.join(save_dir, "finetuned.pt"))
    
    # Log metrics to WandB
    if config.use_wandb:
        wandb.log({
            "target_id": i,
            "val_loss": min(result['history']['val_loss']),
            "train_loss": min(result['history']['train_loss']),
            "epoch": len(result['history']['val_loss'])
        })


print("Fine-tuning complete.")
print(f"Sample history for target 0: {histories[0]}")

Fine-tuning target 0/423...


2025-10-02 15:28:25,617 - INFO - Starting training for 50 epochs...
2025-10-02 15:28:25,618 - INFO - Training samples: 1021
2025-10-02 15:28:25,619 - INFO - Validation samples: 0
2025-10-02 15:28:25,862 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:26,229 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:26,507 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:26,796 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:27,070 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:27,373 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:27,679 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:28:27,987 - INFO - last_patch_pred shape: torch.Size([32]), target shape

Fine-tuning target 1/423...


2025-10-02 15:35:32,398 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:32,678 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:32,990 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:33,275 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:33,579 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:33,862 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:34,159 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:34,473 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:34,775 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:35:35,080 - INFO - last_patch_pred shape:

Fine-tuning target 2/423...


2025-10-02 15:42:37,885 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:38,174 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:38,470 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:38,777 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:39,082 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:39,388 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:39,681 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:39,990 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:40,297 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:42:40,601 - INFO - last_patch_pred shape:

Fine-tuning target 3/423...


2025-10-02 15:47:40,197 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:40,469 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:40,756 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:41,061 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:41,355 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:41,651 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:41,929 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:42,221 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:42,517 - INFO - last_patch_pred shape: torch.Size([32]), target shape: torch.Size([32])
2025-10-02 15:47:42,816 - INFO - last_patch_pred shape: